##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search re-ranking using Gemini embeddings

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   The user will make a search query.
2.   You will use Wikipedia API to return the relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity.
4.   The most relevant search result will be returned as the final answer.

> The non-source code materials in this notebook are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/legalcode.

## Setup


In [ ]:
%pip install -q -U "google-genai>=1.0.0"

In [ ]:
%pip install -q wikipedia

  Preparing metadata (setup.py) ... done


Note: The [`wikipedia` package](https://pypi.org/project/wikipedia/) notes that it was "designed for ease of use and simplicity, not for advanced use", and that production or heavy use should instead "use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)".

In [ ]:
import json
import textwrap

from google import genai
from google.genai import types

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

import numpy as np

from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize the client with your API key
client = genai.Client(api_key=GOOGLE_API_KEY)

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

### Define the search function

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-2.0-flash` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [ ]:
def wikipedia_search(search_queries: list[str]) -> list[str]:
  """Search wikipedia for each query and summarize relevant docs."""
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []
  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data using the client with gemini-2.0-flash model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content

        # Using the client to generate content with the new SDK
        response = client.models.generate_content(
            model='gemini-2.0-flash',
            contents=textwrap.dedent(f"""\
                Extract relevant information
                about user's query: {query}
                From this source:

                {page}

                Note: Do not summarize. Only Extract and return the relevant information
            """)
        )

        urls = [url]
        # In the new SDK, citation metadata handling has changed
        # Check if response has citation metadata and handle it properly
        try:
          if response.candidates and response.candidates[0].citation_metadata:
            # In the new SDK, citation sources may be accessed differently
            # Check the structure of the citation_metadata
            if hasattr(response.candidates[0].citation_metadata, 'citation_sources'):
              extra_citations = response.candidates[0].citation_metadata.citation_sources
              extra_urls = [source.url for source in extra_citations]
              urls.extend(extra_urls)
              search_urls.extend(extra_urls)
              print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        except AttributeError as e:
          print(f"Error processing citation metadata: {str(e)}")
          # Continue processing without citations
          pass

        try:
          text = response.text
        except (ValueError, AttributeError) as e:
          print(f"Error processing response text: {str(e)}")
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")
        continue

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

      except Exception as e:
        print(f'Error processing {search_term}: {str(e)}')
        continue

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results



In [ ]:
example = wikipedia_search(["What are LLMs?"])

Searching for "What are LLMs?"
Related search terms: ['Large language model', 'Retrieval-augmented generation', 'DeepSeek']
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Fetching page: "Retrieval-augmented generation"
Information Source: https://en.wikipedia.org/wiki/Retrieval-augmented_generation
Fetching page: "DeepSeek"
Information Source: https://en.wikipedia.org/wiki/DeepSeek
Information Sources:
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/Retrieval-augmented_generation
     https://en.wikipedia.org/wiki/DeepSeek


Here is what the search results look like:

In [ ]:
from IPython.display import display

for e in example:
  display(to_markdown(e))

> *   **Definition:** A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. They are language models with many parameters, and are trained with self-supervised learning on a vast amount of text.
> *   **Types:** The largest and most capable LLMs are generative pretrained transformers (GPTs).
> *   **Capabilities:** They can be fine-tuned for specific tasks or guided by prompt engineering. These models acquire predictive power regarding syntax, semantics, and ontologies inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained in.
> *   **Training:** LLMs are trained on a vast amount of text using self-supervised learning.
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Large_language_model

> *   **Definition:** LLMs (Large Language Models) are generative artificial intelligence models that can be modified using Retrieval-Augmented Generation (RAG) to respond to user queries with reference to a specified set of documents.
> *   **Traditional LLMs vs. RAG-enhanced LLMs:** Traditional LLMs rely on static training data. RAG improves LLMs by incorporating information retrieval before generating responses, pulling relevant text from databases, uploaded documents, or web sources.
> *   **RAG Benefit:** RAG enables LLMs to retrieve and incorporate additional information before generating responses. Unlike LLMs that rely solely on pre-existing training data, RAG integrates newly available data at query time.
> *   **Limitation:** LLMs aren't humans, their training data can age quickly. LLMs often can’t distinguish specific sources of its knowledge, as all its training data is blended together.
> *   **RAG and LLM Limitations** While RAG improves the accuracy of large language models (LLMs), it does not eliminate all challenges. One limitation is that while RAG reduces the need for frequent model retraining, it does not remove it entirely. Additionally, LLMs may struggle to recognize when they lack sufficient information to provide a reliable response.
> * **Hallucinations:** RAG is not a complete solution to the problem of hallucinations in LLMs.
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Retrieval-augmented_generation

> LLMs (Large Language Models) are artificial intelligence models that DeepSeek develops. DeepSeek-R1 provides responses comparable to other LLMs, such as OpenAI's GPT-4o and o1. DeepSeek's models are described as "open weight," meaning the exact parameters are openly shared, although certain usage conditions differ from typical open-source software.
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/DeepSeek

### Pass the tools to the model

In the new Google GenAI SDK v1.0.0+, function calling is handled by registering your tools with the client and then providing them when sending messages. The client will handle the extraction of schema from function signatures, and the model may return a function call object when it wants to call the function.

The functions need to have proper type annotations for parameters and return values to work correctly.

## Generate supporting search queries

In order to have multiple supporting search queries to the user's original query, you will ask the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [ ]:
instructions = """You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to  the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

Here is the user's query: {query}
"""

In order to yield creative and a more random variety of questions, you will set the model's temperature parameter to a value higher. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

## Enable automatic function calling and call the API

With the new Gen AI SDK, you no longer need to pass `enable_automatic_function_calling=True`. As soon as you attach your function in `tools` on the `GenerateContentConfig`, calling `chat.send_message(...)` will:

1. Invoke your `wikipedia_search` function under the hood
2. Inject its output back into the model
3. Return the final assistant reply—all in one step, without any extra boilerplate.

In [ ]:
# # Create a chat session with the client with configuration
# Attach your function tool here in a GenerateContentConfig
chat = client.chats.create(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.6,
        tools=[wikipedia_search],
    )
)

query = "Explain how deep-sea life survives."

# Now you can just send the message; the chat session will handle
res = chat.send_message(
    message=instructions.format(query=query)
)

Searching for "deep-sea life adaptations"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea gigantism']
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Fetching page: "Deep-sea gigantism"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_gigantism
Searching for "deep-sea ecosystems"
Related search terms: ['Deep-sea community', 'Deep-sea gigantism', 'Deep sea mining']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Deep sea mining"
Information Source: https://en.wikipedia.org/wiki/Deep_sea_mining
Searching for "hydrothermal vent life"
Related search terms: ['Hydrothermal vent', 'Hydrothermal vent microbial communities', 'Endeavour Hydrothermal Vents']
Fetching page: "Hydrothermal vent"
Information Source: https://en.wikipedia.org/wiki/Hydrothermal_vent
Fetching pag

In [ ]:
to_markdown(res.text)

> Deep-sea life survives in a harsh environment of low temperatures, darkness, and immense pressure through a variety of adaptations. Here's a breakdown:
> 
> *   **Energy Sources:** Since sunlight doesn't penetrate the deep sea, many organisms rely on chemosynthesis, a process where bacteria use chemicals like sulfide from hydrothermal vents to produce energy. Others depend on marine snow (organic matter sinking from upper waters), occasional surface blooms, and whale falls (carcasses).
> *   **Adaptations to Pressure:** Deep-sea organisms have developed adaptations in their proteins, anatomical structures, and metabolic systems to withstand high pressure. They increase the proportion of unsaturated fatty acids in their cell membranes to maintain fluidity. Some have also developed pressure tolerance through changes in the mechanism of their α-actin. Specific osmolytes like Trimethylamine N-oxide (TMAO) protect proteins from high hydrostatic pressure.
> *   **Sensory Adaptations:** Many deep-sea fish are blind and rely on senses like pressure and smell. Those with sight have large, sensitive eyes to detect bioluminescent light.
> *   **Camouflage:** Many species are dark-colored to blend in, while some use counter-illumination (bioluminescence on their bellies) to eliminate their shadows.
> *   **Bioluminescence:** Many deep-sea creatures use bioluminescence for finding food, attracting prey, communication, and defense.
> *   **Buoyancy:** Deep-sea fish often lack swim bladders and have jelly-like flesh, high fat content, and reduced skeletal weight to help with buoyancy.
> *   **Feeding Adaptations:** Some have long feelers to locate prey, while others, like anglerfish, use bioluminescent lures. They may also have adaptations for consuming large prey, such as great teeth, hinged jaws, and expandable bodies.
> *   **Deep-Sea Gigantism:** Some deep-sea animals tend to be larger than their shallow-water relatives, possibly due to colder temperatures, food scarcity, reduced predation pressure, and increased dissolved oxygen.
> *   **Chemosynthesis at Hydrothermal Vents:** At hydrothermal vents, chemosynthetic bacteria form the base of the food chain, supporting diverse communities of tube worms, clams, and other organisms. These bacteria convert heat, methane, and sulfur compounds into energy. Symbiotic relationships are common, where bacteria live inside the tissues of animals, providing them with nutrients.


Check for additional citations:

In [ ]:
res.candidates[0].citation_metadata or 'No citations found'

'No citations found'

That looks like it worked. You can go through the chat history using `get_history()` function to see the details of what was sent and received in the function calls:

In [ ]:
for content in chat.get_history():
  print(f'{content.role} -> ')

  if hasattr(content, 'parts') and content.parts:
    part = content.parts[0]
    if hasattr(part, 'function_call') and part.function_call:
      print(f"Function Call: {part.function_call.name}")
      print(f"Arguments: {json.dumps(part.function_call.args, indent=2)}")
    elif hasattr(part, 'function_response') and part.function_response:
      print(f"Function Response: {part.function_response.name}")
      print(f"Response: {json.dumps(part.function_response.response, indent=2)}")
    else:
      print(content.parts[0].text if hasattr(content.parts[0], 'text') else content.parts[0])
  print('---' * 20)


user -> 
You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to  the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

He

In the chat history you can see all the steps of the conversation:

1. The user sent the query.
2. The model replied with a function call to `wikipedia_search` with a number of relevant searches.
3. Because you configured the tool and provided it when sending the message, the client handled executing the search function and returning the list of article summaries to the model.
4. Following the instructions in the prompt, the model generated a final answer based on those summaries.


## **[Optional]** Manually execute the function call

To inspect and run the model’s function call yourself, disable automatic invocation on the initial message:

* Create your chat session as usual (no tools attached at chat-creation time).
* When sending your prompt, pass a `GenerateContentConfig` with:

  * `tools=[wikipedia_search]`
  * `automatic_function_calling={'disable': True}`

The model will return a raw `FunctionCall` object. You can then extract its `name` and `args`, execute `wikipedia_search(**args)` locally, and finally feed the result back into the chat with another `send_message` to get the completed response.


In [ ]:
# Create a new chat session without automatic function calling
chat = client.chats.create(
    model='gemini-2.0-flash',
    config=types.GenerationConfig(temperature=0.6)
)


In [ ]:
# Send a message to the chat without automatic function calling
result = chat.send_message(
    message=instructions.format(query=query),
    config=types.GenerateContentConfig(
        tools=[wikipedia_search],
        automatic_function_calling={'disable': True},
    )
)

Initially the model returns a FunctionCall:

In [ ]:
# Extract the function call from the response
if result.candidates and len(result.candidates) > 0 and result.candidates[0].content.parts:
    function_call = result.candidates[0].content.parts[0].function_call
    if function_call:
        print(json.dumps({
            "name": function_call.name,
            "args": function_call.args
        }, indent=2))
    else:
        print("No function call found in the response.")
else:
    print("Unexpected response format.")

{
  "name": "wikipedia_search",
  "args": {
    "search_queries": [
      "deep-sea life adaptations",
      "deep-sea ecosystems",
      "hydrothermal vent ecosystems",
      "deep-sea food web",
      "how do deep-sea creatures get energy?",
      "deep-sea organisms survival strategies",
      "challenges of deep-sea environment",
      "deep-sea chemosynthesis",
      "deep-sea pressure adaptation",
      "deep-sea temperature adaptation"
    ]
  }
}


In [ ]:
# Access the function name
if function_call and hasattr(function_call, 'name'):
    function_name = function_call.name
    print(f"Function Name: {function_name}")
else:
    print("Could not extract function name from the function call.")

Function Name: wikipedia_search


Call the function with generated arguments to get the results.

In [ ]:
# Execute the function with the arguments from the function call
if function_call and hasattr(function_call, 'args'):
    summaries = wikipedia_search(**function_call.args)
else:
    print("Could not execute function, missing function call or arguments.")
    summaries = []

Searching for "deep-sea life adaptations"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea gigantism']
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Fetching page: "Deep-sea gigantism"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_gigantism
Searching for "deep-sea ecosystems"
Related search terms: ['Deep-sea community', 'Deep-sea gigantism', 'Deep sea mining']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Deep sea mining"
Information Source: https://en.wikipedia.org/wiki/Deep_sea_mining
Searching for "hydrothermal vent ecosystems"
Related search terms: ['Hydrothermal vent', 'Axial Seamount', 'Hydrothermal vent microbial communities']
Fetching page: "Hydrothermal vent"
Information Source: https://en.wikipedia.org/wiki/Hydrothermal_vent
Fetching page: "Axia

Now send the `FunctionResult` to the model.

In [ ]:
# Build the Part from your function’s output
func_resp_part = types.Part.from_function_response(
    name="wikipedia_search",
    response={"result": summaries},
)

# Send it back into the chat with a GenerateContentConfig
response = chat.send_message(
    message=func_resp_part,
    config=types.GenerateContentConfig()
)

# Render as markdown
to_markdown(response.text)

> Deep-sea life survives through a variety of adaptations to the extreme conditions of its environment. These adaptations can be broadly categorized as relating to energy acquisition, pressure, temperature, and sensory perception.
> 
> *   **Energy Acquisition:** Since sunlight doesn't penetrate the deep sea, photosynthesis is not possible. Deep-sea creatures rely on:
>     *   **Marine snow:** This is organic matter that drifts down from the photic zone above.
>     *   **Scavenging:** Many deep-sea organisms are scavengers, feeding on carcasses that sink to the ocean floor. Whale falls, for example, provide a substantial source of nutrients.
>     *   **Chemosynthesis:** Around hydrothermal vents and cold seeps, chemosynthetic bacteria and archaea form the base of the food chain. These organisms use chemicals like hydrogen sulfide, methane, and other hydrocarbons to produce energy, supporting diverse communities of organisms like tube worms, clams, and shrimp.
>     *   **Predation:** Many deep-sea creatures are predators, adapted to finding and capturing prey in the dark. Some, like the anglerfish, use bioluminescent lures to attract prey.
> 
> *   **Pressure Adaptation:** The immense pressure of the deep sea (increasing by about 1 atmosphere every 10 meters) requires specific adaptations:
>     *   Organisms maintain the same pressure inside their bodies as the external pressure.
>     *   Cellular membranes favor phospholipid bilayers with a higher proportion of unsaturated fatty acids to maintain fluidity.
>     *   Some species have jelly-like flesh made of glycosaminoglycans for low density.
>     *   Reduced or minimal bone structure and the elimination of excess cavities.
> 
> *   **Temperature Adaptation:** The deep sea is generally very cold (around 5-6°C), though hydrothermal vents can have extremely high temperatures:
>     *   Psychrophiles are organisms which thrive in low temperatures (−20 °C to 20 °C). They protect from freezing through ice-induced desiccation and vitrification.
>     *   Lipid membrane adaptation with high content of short, unsaturated fatty acids and carotenoids to maintain membrane fluidity.
>     *   Antifreeze proteins: Prevent ice formation and recrystallization.
> 
> *   **Sensory Adaptations:** In the absence of sunlight, deep-sea creatures have developed alternative sensory strategies:
>     *   **Bioluminescence:** Many species use bioluminescence for attracting prey, communication, and camouflage (counter-illumination).
>     *   **Large, sensitive eyes:** Some fish have large, tubular eyes adapted to detecting the faint bioluminescent light. Some eyes are even 100 times more sensitive than human eyes.
>     *   **Other senses:** Some species rely on sensitivity to pressure changes and smell to locate prey and mates.
> 
> *   **Other Adaptations:**
>     *   **Buoyancy:** Many deep-sea creatures lack swim bladders due to the high pressure. They may have jelly-like flesh, high fat content, and reduced skeletal weight to maintain buoyancy.
>     *   **Feeding:** Adaptations for consuming large prey include sharp teeth, hinged jaws, large mouths, and expandable bodies, as seen in the gulper eel.
>     *   **Gigantism:** Some deep-sea organisms exhibit deep-sea gigantism, growing larger than their shallow-water relatives, potentially as an adaptation to colder temperatures, food scarcity, and reduced predation pressure.
>     *   **Reproduction:** Some organisms are hermaphroditic due to the difficulty of finding mates in the vast, dark environment.
> 
> These diverse adaptations allow life to thrive in the extreme and unique environment of the deep sea.


## Re-ranking the search results

Helper function to embed the content:

In [ ]:
def get_embeddings(texts: list[str]) -> np.ndarray:
    # call the new embed_content endpoint
    response = client.models.embed_content(
        model='text-embedding-004',   # updated model ID
        contents=texts
    )
    # extract the vector for each input
    embeddings = [e.values for e in response.embeddings]
    return np.array(embeddings)

Please refer to the [embeddings guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement cosine similarity as your metric. Here returned embedding vectors will be of unit length and hence their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating cosine similarity is esentially same as calculating their dot product score.

In [ ]:
def dot_product(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return a @ b.T

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [ ]:
search_res = get_embeddings(summaries)
query_emb  = get_embeddings([query])

Calculate similarity score:

In [ ]:
sim_value = dot_product(search_res, query_emb)

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [ ]:
print(summaries[np.argmax(sim_value)])

Deep-sea organisms survival strategies (specifically related to cold temperatures, based on the context of the source):

*   **Psychrophiles:** Organisms that thrive in low temperatures (−20 °C to 20 °C) and are found in the deep sea.
*   **Adaptations to cold environments:**
    *   **Protection from freezing:** Ice-induced desiccation and vitrification (glass transition).
    *   **Lipid membrane adaptation:** High content of short, unsaturated fatty acids and carotenoids to maintain membrane fluidity.
    *   **Antifreeze proteins:** Prevent ice formation and recrystallization.
    *   **Enzyme flexibility:** Increased flexibility of enzyme structure to compensate for freezing.
    *   **Viable but Nonculturable (VBNC) state:** Transition into a state where the organism can respire and use substrates but cannot replicate, potentially as a survival strategy.
*   **High pressure:** Adaptation to high pressure in the deep sea.
*   **Taxonomic range:** bacteria
*   **Psychrotrophic micr

### Similarity with Hypothetical Document Embeddings (HyDE)

Drawing inspiration from [Gao et al](https://arxiv.org/abs/2212.10496) the objective here is to generate a template answer to the user's query using `gemini-2.0-flash`'s internal knowledge. This hypothetical answer will serve as a baseline to calculate relevance of all the search results.

In [ ]:
# Generate hypothetical answer using the client
res = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=f"""Generate a hypothetical answer
to the user's query by using your own knowledge. Assume that you know everything
about the said topic. Do not use factual information, instead use placeholders
to complete your answer. Your answer should feel like it has been written by a human.

query: {query}"""
)

to_markdown(res.text)

> Alright, let's dive deep (pun intended!) into the fascinating world of deep-sea survival. Forget sunshine and gentle breezes – down there, it's a whole different ballgame. It's a world defined by crushing pressure, perpetual darkness, and a scarcity of food. So, how do these incredible creatures manage to thrive? It's a symphony of adaptations, cleverly tuned to the unique challenges of their environment.
> 
> Firstly, **Pressure, Pressure Everywhere**: We’re talking pressures that would turn a human into a… well, let’s just say something resembling a pancake. Deep-sea creatures have evolved several strategies to cope. Many don’t have air-filled cavities like lungs or swim bladders, which would be crushed. Instead, their bodies are often filled with fluids and oils that are nearly incompressible. Think of it like comparing a balloon to a water balloon – which one handles pressure better? Exactly! They also have specialized proteins and enzymes that function efficiently under immense pressure, a bit like having tiny, pressure-resistant power tools.
> 
> Secondly, **In Total Darkness**: The sun doesn’t penetrate this far, so photosynthesis is out. That means plants are non-existent. The deep sea is largely dependent on the "marine snow," which is organic detritus that rains down from the surface waters. But, that’s not always enough! Some species are masters of *[Hypothetical feeding adaptation based on specific deep-sea organism]*, while others rely on **chemosynthesis**, using chemicals like hydrogen sulfide or methane spewed from hydrothermal vents as a source of energy. Imagine them as underwater factories, churning out energy from the Earth itself!
> 
> Thirdly, **Food – A Scarcity Mindset**: Since food is limited, deep-sea creatures have evolved incredibly efficient metabolisms. They're essentially running on fumes, moving slowly and conserving energy wherever possible. Many are ambush predators, lying in wait for unsuspecting prey to wander by. Picture a *[Hypothetical deep-sea predator]* with its *[Hypothetical lure adaptation]*, patiently waiting in the blackness for its next meal. Others have developed huge mouths and expandable stomachs, allowing them to capitalize on the rare opportunities for a big meal. Think "feast or famine" on steroids!
> 
> Finally, **Communication and Reproduction**: Finding a mate in the vast darkness can be a challenge. Many deep-sea creatures use **bioluminescence**, producing their own light to attract partners (or prey!). Think of them as living Christmas lights, twinkling in the abyssal plains. They also use *[Hypothetical method of sensing environment and finding mates in the dark]* to communicate and navigate. Reproduction strategies are equally fascinating, with some species exhibiting unusual mating behaviors and *[Hypothetical reproduction strategy]*.
> 
> In short, deep-sea life is a testament to the power of adaptation. These creatures have evolved remarkable strategies to overcome the extreme challenges of their environment, demonstrating the incredible resilience and ingenuity of life on Earth. It's a world of weird and wonderful creatures, constantly surprising us with their bizarre and beautiful adaptations.


Use Gemini API to get embeddings for the baseline answer and compare them with search results

In [ ]:
hypothetical_ans = get_embeddings([res.text])

Calculate similarity scores to rank the search results

In [ ]:
sim_value = dot_product(search_res, hypothetical_ans)

In [ ]:
sim_value

array([[0.82175196],
       [0.83640874],
       [0.77970876],
       [0.79748942],
       [0.7002488 ],
       [0.71700249],
       [0.64705747],
       [0.68503258],
       [0.61503797],
       [0.51633416],
       [0.63575328],
       [0.67186001],
       [0.66982765],
       [0.7761045 ]])

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [ ]:
to_markdown(summaries[np.argmax(sim_value)])

> **General Adaptations to Deep-Sea Life:**
> 
> *   **Sensory Adaptations:**
> 
>     *   Blindness in some species, relying on sensitivity to pressure changes and smell.
>     *   Large, sensitive eyes in others, adapted to using bioluminescent light, up to 100 times more sensitive than human eyes.
>     *   Multiple Rh1 opsin genes (e.g., silver spinyfin with 38) for enhanced vision in dim light.
> *   **Camouflage:** Dark coloration to blend in with the environment and avoid predation.
> *   **Bioluminescence:**
> 
>     *   Common adaptation in deep-sea fish.
>     *   Production of light through luciferin molecules and oxygen.
>     *   Photophores: light-producing glandular cells with luminous bacteria, sometimes with lenses.
>     *   Purposes: attracting prey (anglerfish), claiming territory, communication/mate finding, distracting predators.
>     *   Counter-illumination: Camouflaging from predators by illuminating bellies to match light from above.
> *   **Buoyancy:**
> 
>     *   Lack of swim bladders due to high pressure.
>     *   Hydrofoil-like structures for hydrodynamic lift.
>     *   Jelly-like flesh and minimal bone structure to reduce tissue density.
>     *   High fat content, reduced skeletal weight, and water accumulation for buoyancy.
> *   **Feeding:**
> 
>     *   Long feelers to locate prey/mates in darkness.
>     *   Anglerfish with bioluminescent lure.
>     *   Adaptations for consuming large prey: sharp teeth, hinged jaws, large mouths, expandable bodies (gulper eel).
> *   **Environment**
> 
>     * Marine snow provides energy.
> * **Pressure Adaptation:**
>     * Deep-sea organisms have the same pressure within their bodies as is exerted on them from the outside.
>     * Reduced fluidity of membranes.
>     * Increasing the proportion of unsaturated fatty acids in the lipids of the cell membranes.
>     * Biochemical reactions must ultimately decrease the volume of the organism to some degree.
> 
> **Specific Adaptations in Different Zones:**
> 
> *   **Mesopelagic Fish:**
> 
>     *   Daily vertical migrations.
>     *   Muscular bodies, ossified bones, scales, developed gills and nervous systems, large hearts and kidneys.
>     *   Plankton feeders have small mouths with fine gill rakers; piscivores have larger mouths and coarser gill rakers.
>     *   Large eyes as visual predators.
>     *   Tubular eyes with big lenses looking upwards for improved terminal vision.
>     *   Lack of defensive spines, use color for camouflage.
>     *   Ambush predators are dark/black/red; migratory forms use silvery colors.
>     *   Photophores for counter-illumination.
> *   **Bathypelagic Fish:**
> 
>     *   Slow metabolisms and unspecialized diets.
>     *   Lie-in-wait predators.
>     *   Elongated bodies with weak muscles and skeletal structures.
>     *   Extensible, hinged jaws with recurved teeth.
>     *   Slimy, without scales.
>     *   Reduced or non-functional eyes.
>     *   Small or missing gills, kidneys, hearts, and swim bladders.
>     *   Black or red coloration with few photophores.
>     *   Hermaphroditism in some species to increase chances of mating.
>     *   Male anglerfish fusing with females for reproduction.
> *   **Pressure adaptation:**
>     * Deep-sea organisms possess adaptations at cellular and physiological levels that allow them to survive in environments of great pressure.
>     * Equilibrium of chemical reactions is disturbed by pressure, and pressure can inhibit processes which result in an increase in volume.
>     * Enzymatic reactions that induce changes in water organization effectively change the system's volume.
>     * Deep-sea cellular membranes favor phospholipid bilayers with a higher proportion of unsaturated fatty acids, which induce a higher fluidity than their sea-level counterparts.
> * **Lantern fish:**
>     * Account for as much as 65% of all deep-sea fish biomass.
>     * Playing an important ecological role as prey for larger organisms.
> * **Endangered Species**
>     * Slow reproduction of these fish
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Deep-sea_fish

You have now created a search re-ranking engine using Gemini embeddings with the Google GenAI SDK v1.0.0+!

In this notebook, you've learned how to:
1. Set up and use the new Google GenAI client
2. Use function calling to access the Wikipedia API
3. Embed content using the new embedding API format
4. Re-rank search results using similarity metrics

## Next steps

I hope you found this example helpful! Check out more examples in the [Gemini Guide](https://github.com/google-gemini/gemini-guide/) to learn more.